# Problem Set 3
## Importing and looking at the data

In [2]:
import pandas as pd
import numpy as np
import scipy.optimize as opt
import math

lab_data = pd.read_stata('PS3_data.dta')
lab_data.head(n=5)

,id68,year,intid,relhh,hannhrs,wannhrs,hlabinc,wlabinc,nochild,wrace,...,redpregovinc,hsex,wsex,age,wage,hpersno,wpersno,hyrsed,wyrsed,pce
0,1,1967,1,Head,1200.0,2000.0,NaN,NaN,0,NaN,...,5614.0,1.0,2.0,52.0,46.0,1.0,2.0,8.0,8.0,0.0
1,2,1967,2,Head,0.0,0.0,NaN,NaN,0,NaN,...,0.0,1.0,2.0,56.0,57.0,1.0,2.0,3.0,3.0,0.0
2,3,1967,3,Head,0.0,0.0,NaN,NaN,0,NaN,...,0.0,1.0,2.0,77.0,64.0,1.0,2.0,NaN,3.0,0.0
3,4,1967,4,Head,1560.0,0.0,NaN,NaN,6,1.0,...,3280.0,1.0,2.0,45.0,44.0,1.0,2.0,8.0,5.0,0.0
4,5,1967,5,Head,2500.0,2000.0,NaN,NaN,3,1.0,...,7900.0,1.0,2.0,24.0,22.0,1.0,2.0,10.0,9.0,0.0


In [3]:
lab_data.describe()

,id68,year,intid,hannhrs,wannhrs,hlabinc,wlabinc,nochild,wrace,hrace,...,redpregovinc,hsex,wsex,age,wage,hpersno,wpersno,hyrsed,wyrsed,pce
count,123786.000000,123786.000000,123786.000000,123786.000000,123786.000000,9.023300e+04,48496.000000,123786.000000,90603.000000,123656.000000,...,1.237860e+05,123786.000000,80758.0,123786.000000,80758.000000,123786.000000,80758.000000,122809.000000,80091.000000,123786.000000
mean,1494.639475,1984.831273,3271.379429,1679.269897,633.026917,4.211505e+04,22026.289062,0.843771,1.098220,1.129731,...,3.012258e+04,1.233072,2.0,45.545547,41.390785,39.620201,55.346169,12.666091,12.720081,0.557690
std,838.901790,9.836212,2277.056058,1061.704712,878.422791,4.670424e+04,21336.107422,1.182829,0.356161,0.394627,...,4.588795e+04,0.422940,0.0,17.623671,14.786721,69.003265,77.864296,2.917721,2.422607,0.265198
min,1.000000,1967.000000,1.000000,0.000000,0.000000,6.353981e-01,1.192780,0.000000,1.000000,1.000000,...,-1.324040e+05,1.000000,2.0,16.000000,13.000000,1.000000,1.000000,1.000000,1.000000,0.000000
25%,772.000000,1977.000000,1444.000000,832.000000,0.000000,1.979858e+04,8016.247070,0.000000,1.000000,1.000000,...,7.700000e+03,1.000000,2.0,31.000000,29.000000,1.000000,2.000000,12.000000,12.000000,0.362158
50%,1517.000000,1985.000000,2984.000000,1976.000000,0.000000,3.460022e+04,18122.412109,0.000000,1.000000,1.000000,...,1.900000e+04,1.000000,2.0,42.000000,39.000000,3.000000,3.000000,12.000000,12.000000,0.599887
75%,2224.000000,1993.000000,4763.000000,2350.000000,1454.000000,5.267309e+04,30256.060547,2.000000,1.000000,1.000000,...,3.910775e+04,1.000000,2.0,58.000000,51.000000,22.000000,170.000000,15.000000,14.000000,0.786908
max,2930.000000,2002.000000,16968.000000,7800.000000,5840.000000,3.771521e+06,856942.062500,11.000000,8.000000,8.000000,...,3.660000e+06,2.000000,2.0,102.000000,95.000000,227.000000,231.000000,17.000000,17.000000,0.928007


## Clean the data
### Question 1

In [4]:
# Clean the data
# Limit the sample to only male heads
lab_data = lab_data[ lab_data.hsex == 1 ]

# Define the min and max ages and sample years
minAge = 25
maxAge = 60
years = [1971, 1980, 1990, 2000]

# Limit to only those making > $7/hr
minWage = 7

### Question 2

In [5]:
# Create dummy variables for race
lab_data['Black'] = (lab_data['hrace'] == 2).astype(int)
lab_data['Hispanic'] = (lab_data['hrace'] == 5).astype(int)
lab_data['OtherRace'] = ((lab_data['hrace'] != 1) & (lab_data['hrace'] != 2) & (lab_data['hrace'] != 5)).astype(int)

# Create log wage variable
lab_data['hwage'] = lab_data['hlabinc'] / lab_data['hannhrs']
lab_data['lwage'] = np.log(lab_data['hwage'])

# Keep only the variables we need
lab_data = lab_data[ ['lwage', 'hwage', 'hlabinc', 'hannhrs', 'hyrsed', 'age', 'Black', 'Hispanic', 'OtherRace', 'year' ] ]

# Limit the data to just the relevant values
lab_data = lab_data[ (lab_data.hwage > minWage) & (lab_data.age >= minAge) & \
                    (lab_data.age <= maxAge) & (lab_data.lwage != np.inf)]
lab_data['cons'] = 1

In [6]:
lab_data.describe() # Check the data to make sure I have the correct variables

,lwage,hwage,hlabinc,hannhrs,hyrsed,age,Black,Hispanic,OtherRace,year,cons
count,57477.000000,57477.000000,5.747700e+04,57477.000000,57097.000000,57477.000000,57477.000000,57477.0,57477.000000,57477.000000,57477.0
mean,3.010414,24.306034,5.280176e+04,2228.364990,13.529993,39.224247,0.056388,0.0,0.024706,1986.635245,1.0
std,0.543891,25.154028,5.228071e+04,619.743286,2.449510,9.579065,0.230671,0.0,0.155228,8.744894,0.0
min,1.945946,7.000252,1.666980e+01,2.000000,1.000000,25.000000,0.000000,0.0,0.000000,1971.000000,1.0
25%,2.635309,13.947624,3.035003e+04,1952.000000,12.000000,31.000000,0.000000,0.0,0.000000,1979.000000,1.0
50%,2.990979,19.905161,4.377034e+04,2160.000000,13.000000,38.000000,0.000000,0.0,0.000000,1987.000000,1.0
75%,3.324576,27.787226,6.136955e+04,2517.000000,16.000000,47.000000,0.000000,0.0,0.000000,1994.000000,1.0
max,7.448526,1717.330322,3.771521e+06,5840.000000,17.000000,60.000000,1.000000,0.0,1.000000,2002.000000,1.0


## Log Likelihood Estimator

In [7]:
# Define function to compute the log likelihood
def model_ML(b, data, t):
    '''
    Compute the log likelihood
    
    Args:
        B: a vector of coefficient values
        sigma (float): the standard deviation of the error term of the model
        data: a dataframe of the data to be used
        t: the year(s) of interest
        
    Returns:
        negLL (float): the negative Log Likelihood
    '''
    mdata = data[data['year'].isin(t)]
    # Did not include a coefficient for 'Hispanic' b/c there were no observations, so it was not used in the model
    const = b[0]
    B1 = b[1]
    B2 = b[2]
    B3 = b[3]
    B4 = b[4]
    sigma = b[5]
    
    N = len(mdata['lwage'])
    
    # The linear model
    lnwage = const + B1 * mdata['hyrsed'] + B2 * mdata['age'] + B3 * mdata['Black'] + B4 * mdata['OtherRace']
    
    # The error term for getting Log Likelihood
    e = mdata['lwage'] - lnwage
    
    # Calculate negative Log Likelihood
    negLL = (N/2) * np.log(sigma ** 2) + (N/2) * np.log(2 * math.pi) + (np.sum(e ** 2)) / (2 * sigma ** 2)
    
    return(negLL)

### Question 3

In [19]:
guess = [1, 0.6, 0, 0.5, 1, 0.5] # initial coefficient guesses
Beta = []
for t in years:
    params = (lab_data, [t])
    results = opt.minimize(model_ML, guess, args = params , method = 'Nelder-Mead', tol = 1e-10, options = {'maxiter': 5000})
    Beta.append(results.x)
    print(t, results.x)

1971 [ 1.54620963  0.06695837  0.01448222 -0.16320529  0.03303406  0.40737389]
1980 [ 1.61314887  0.06755533  0.01269698 -0.10267612  0.01370988  0.44912802]
1990 [ 1.29900318  0.10165807  0.00743782 -0.21626832 -0.15542311  0.48425843]
2000 [ 1.16178352  0.10920915  0.01097248 -0.24618434 -0.05874108  0.53158434]


In [18]:
guess = [1, -4, 0, 0.5, 1, 0.5] # initial coefficient guesses
bnds = ((None, None), (None, None), (None, None), (None, None), (None, None), (0,None))
Beta = []
for t in years:
    params = (lab_data, [t])
    results = opt.minimize(model_ML, guess, args = params , method = 'L-BFGS-B', bounds = bnds, options = {'maxiter': 5000})
    Beta.append(results.x)
    print(t, results.x)

1971 [  1.          -4.           1.36564673 -13.41659556  -5.76834306
  20.58698667]


/Users/JonathanT/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:34: RuntimeWarning: divide by zero encountered in log
/Users/JonathanT/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:34: RuntimeWarning: divide by zero encountered in double_scalars
/Users/JonathanT/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:34: RuntimeWarning: invalid value encountered in double_scalars


1980 [ 1.         -4.          1.46680759 -5.32122577 -5.54623823 19.18077199]
1990 [ 1.00000000e+00 -4.00000000e+00  1.70744260e+00 -2.92970038e+00
  8.11099980e-02  3.07064346e+02]
2000 [ 1.         -4.          0.         54.28890075 57.44894963 55.06931056]


### Question 4

The $\beta_1$ coefficient can be interpreted as the percent change in wage as a response to one more year of education. So, for the year 1971, for an additional year of education, a person might expect to earn 6.7% more (using the results from the Nelder-Mead method). Because the outcome variable is log wage, it can be interpreted as a percent change.

It can be seen that the returns to education have been increasing over time. There was a somewhat small increase between 1971 and 1980. Then there was a much larger increase over the next ten years followed by again a slightly smaller increase between 1990 and 2000.

In [21]:
import statsmodels.api as sm

# Check my coefficients from the MLE method against OLS estimation for the year 1971
testDat = lab_data[lab_data.year == 1971]
Y = testDat['lwage']
X = testDat[['cons','hyrsed', 'age', 'Black', 'OtherRace']]
linreg = sm.OLS(endog = Y, exog = X, missing = 'drop')
finresult = linreg.fit()
print('1971 Results: ', finresult.summary())

1971 Results:                              OLS Regression Results                            
Dep. Variable:                  lwage   R-squared:                       0.244
Model:                            OLS   Adj. R-squared:                  0.241
Method:                 Least Squares   F-statistic:                     110.7
Date:                Mon, 30 Sep 2019   Prob (F-statistic):           7.42e-82
Time:                        18:17:10   Log-Likelihood:                -728.06
No. Observations:                1380   AIC:                             1466.
Df Residuals:                    1375   BIC:                             1492.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
cons           1.5510      0.073     

In [23]:
# Check my coefficients from the MLE method against OLS estimation for the year 1980
testDat2 = lab_data[lab_data.year == 1980]
linreg2 = sm.OLS(endog = testDat2['lwage'], exog = testDat2[['cons', 'hyrsed','age','Black','OtherRace']], missing = 'drop')
finresult2 = linreg2.fit()
print('1980 Results: ', finresult2.summary())

1980 Results:                              OLS Regression Results                            
Dep. Variable:                  lwage   R-squared:                       0.169
Model:                            OLS   Adj. R-squared:                  0.167
Method:                 Least Squares   F-statistic:                     94.08
Date:                Mon, 30 Sep 2019   Prob (F-statistic):           6.41e-73
Time:                        18:18:04   Log-Likelihood:                -1148.4
No. Observations:                1856   AIC:                             2307.
Df Residuals:                    1851   BIC:                             2334.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
cons           1.6131      0.075     

In [25]:
# Check my coefficients from the MLE method against OLS estimation for the year 1990
testDat3 = lab_data[lab_data.year == 1990]
linreg3 = sm.OLS(endog = testDat3['lwage'], exog = testDat3[['cons', 'hyrsed','age','Black','OtherRace']], missing = 'drop')
finresult3 = linreg3.fit()
print('1990 Results: ', finresult3.summary())

1990 Results:                              OLS Regression Results                            
Dep. Variable:                  lwage   R-squared:                       0.217
Model:                            OLS   Adj. R-squared:                  0.216
Method:                 Least Squares   F-statistic:                     139.3
Date:                Mon, 30 Sep 2019   Prob (F-statistic):          3.67e-105
Time:                        18:19:14   Log-Likelihood:                -1393.9
No. Observations:                2013   AIC:                             2798.
Df Residuals:                    2008   BIC:                             2826.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
cons           1.1186      0.084     

In [26]:
# Check my coefficients from the MLE method against OLS estimation for the year 2000
testDat4 = lab_data[lab_data.year == 2000]
linreg4 = sm.OLS(endog = testDat4['lwage'], exog = testDat4[['cons', 'hyrsed','age','Black','OtherRace']], missing = 'drop')
finresult4 = linreg4.fit()
print('2000 Results: ', finresult4.summary())

2000 Results:                              OLS Regression Results                            
Dep. Variable:                  lwage   R-squared:                       0.207
Model:                            OLS   Adj. R-squared:                  0.205
Method:                 Least Squares   F-statistic:                     168.7
Date:                Mon, 30 Sep 2019   Prob (F-statistic):          1.66e-128
Time:                        18:19:59   Log-Likelihood:                -2081.3
No. Observations:                2595   AIC:                             4173.
Df Residuals:                    2590   BIC:                             4202.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
cons           1.1619      0.080     